<a href="https://colab.research.google.com/github/gpandu/CodeGenGPT/blob/main/CodeGPT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tokenization:



*   Subword Tokenization : Keep frequent words and break rearer words into subwords
*   A statastical Alogrothm learns how to do this based on corpus.

> Ex: Listeria ---> "more" , "over"

> "more" and "over" are likely to be more frequent than moreover


*   Tokenization has better chance of handling OOV words while decreasing the size of the overall dictionary.   









In [21]:
!pip install transformers
!pip install tokenizers
!pip install datasets

In [22]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.BPE())

In [23]:
import math
import numpy as np
import tensorflow as tf

In [24]:
from datasets import load_dataset

# load code dataset
raw_dataset = load_dataset("code_search_net", "python")

In [25]:
def get_training_corpus():
    dataset = raw_dataset["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]


In [26]:
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)


In [27]:
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [28]:
raw_dataset["train"][1]["whole_func_string"]

'def include(self):\n        """\n        If the next line is an include statement, inserts the contents\n        of the included file into the pending buffer.\n        """\n        if len(self.pending) == 0 or not self.pending[0].startswith(\'include \'):\n            return\n        name = self.pending.pop(0)[8:].strip()[1:-1]\n        for b in [os.path.dirname(self.name)] + self.inc_dirs:\n            pname = os.path.abspath(os.path.expanduser(os.path.join(b, name)))\n            if os.path.isfile(pname):\n                name = pname\n                break\n        else:\n            raise Exception(\'Can not find include file "{}".\'.format(name))\n        self.pending = list(FortranReader(name, self.docmark, self.predocmark, \n                                          self.docmark_alt, self.predocmark_alt,\n                                          self.fixed, self.length_limit,\n                                          inc_dirs=self.inc_dirs)) + self.pending'

In [10]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokenizer.enable_padding(length=256, pad_id = 0, pad_token = "<|endoftext|>")
tokenizer.enable_truncation(max_length=256)
encoding = tokenizer.encode(raw_dataset["train"][1]["whole_func_string"])
print(encoding.ids)
print(encoding.attention_mask)

[295, 2208, 63, 18133, 8, 1386, 12, 1626, 12, 1424, 63, 369, 12, 6330, 29, 325, 274, 232, 290, 10890, 256, 1458, 2009, 18007, 311, 231, 1626, 1424, 461, 231, 495, 1312, 222, 256, 581, 5729, 15, 18133, 14, 1825, 423, 1746, 280, 232, 264, 339, 6426, 63, 2036, 14, 12916, 63, 12083, 63, 976, 4619, 3889, 26, 222, 302, 1197, 18133, 392, 377, 93, 232, 18007, 233, 7306, 14, 2286, 14, 1250, 8, 1747, 29, 1747, 12, 3327, 29, 5820, 12, 1363, 630, 29, 1430, 63, 369, 625, 1572, 63, 876, 3342, 10022, 63, 526, 397, 232, 264, 553, 8, 18133, 9, 659, 554, 26, 222, 302, 1197, 18133, 392, 377, 93, 232, 302, 1197, 18133, 392, 18007, 59, 16, 26, 1328, 6641, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [29]:
def process_train_data(dataset, tokenizer, len):
  tokenizer.enable_padding(length=257, pad_id = 0, pad_token = "<|endoftext|>")
  tokenizer.enable_truncation(max_length=257)
  decoder_inputs = []
  decoder_targets = []
  for i in range(len):
    seqs = tokenizer.encode(dataset[i]["whole_func_string"])
    decoder_inputs.append(seqs.ids[:-1]) # Drop the last token in the sentence.
    decoder_targets.append(seqs.ids[1:]) # Drop the first token in the sentence.
  return decoder_inputs, decoder_targets


In [ ]:
#tf_dataset = raw_dataset.with_format("tf")
#tf.dataset.

In [30]:
decoder_inputs, decoder_targets = process_train_data(raw_dataset["train"], tokenizer, 1000)
print(decoder_inputs[200])
print(decoder_targets[200])

[295, 7964, 63, 2392, 63, 4287, 8, 248, 12, 2375, 1119, 12, 655, 527, 274, 222, 290, 222, 21910, 292, 969, 231, 5579, 5337, 5983, 24877, 11000, 253, 309, 10134, 298, 1169, 26, 241, 2375, 1119, 310, 507, 1554, 311, 256, 10134, 241, 922, 310, 507, 1445, 292, 969, 231, 5579, 5337, 8681, 16699, 6407, 2424, 298, 734, 26, 241, 486, 2368, 14, 1278, 14, 2712, 1814, 256, 1354, 497, 8133, 877, 241, 256, 730, 222, 290, 222, 302, 272, 14, 1244, 8, 241, 2375, 1119, 12, 241, 1444, 29, 4415, 5337, 8681, 16699, 6407, 2424, 2136, 527, 9, 222, 687, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [31]:
tf_dataset = tf.data.Dataset.from_tensor_slices((decoder_inputs, decoder_targets)).batch(10, drop_remainder=True)



Multi Head Attention



*   Each Attention head performs Scaled Dot Product Self-Attention operation where given Keys, Query and Values, the return matrix of values given by below operation.

        Attention(Q,K,V) = softmax((Q*Transpose(K))/sqrt(d))*V





In [32]:
def scaled_dot_product_attention(query, key, value, mask=None):
  key_dims = tf.cast(tf.shape(key)[-1], tf.float32)
  scaled_scores = tf.matmul(query, key, transpose_b=True) / tf.math.sqrt(key_dims)

  if mask is not None:
    scaled_scores = tf.where(mask==0, -np.inf, scaled_scores)

  softmax = tf.keras.layers.Softmax()
  weights = softmax(scaled_scores)
  return tf.matmul(weights, value), weights



In [40]:
output, attn_weights = scaled_dot_product_attention(queries, keys, values)

print("Output\n", output, "\n")
print("Weights\n", attn_weights)

Output
 tf.Tensor(
[[0.51509905 0.492224   0.41562238 ... 0.49542582 0.4560463  0.5025198 ]
 [0.51484996 0.49366552 0.41918206 ... 0.48865992 0.45576534 0.49914658]
 [0.51344013 0.49318826 0.41454804 ... 0.49276724 0.46061277 0.5048734 ]
 ...
 [0.5171826  0.49254543 0.4134746  ... 0.49598488 0.4551604  0.50469285]
 [0.516986   0.4962027  0.41876078 ... 0.49199528 0.45513254 0.501719  ]
 [0.5198245  0.49181268 0.4196138  ... 0.49218264 0.45782724 0.50488156]], shape=(100, 512), dtype=float32) 

Weights
 tf.Tensor(
[[0.00936828 0.0110294  0.01296335 ... 0.01008087 0.00891249 0.0100021 ]
 [0.0074634  0.01211887 0.01438605 ... 0.00902398 0.00853538 0.00794049]
 [0.01058757 0.01182669 0.01492247 ... 0.01171989 0.00783003 0.00828462]
 ...
 [0.01026133 0.01016313 0.01234818 ... 0.01023888 0.00775721 0.00839929]
 [0.00880167 0.01057488 0.01439731 ... 0.00982494 0.00842312 0.00971635]
 [0.00812568 0.00966703 0.01244548 ... 0.01076922 0.00953873 0.00853225]], shape=(100, 100), dtype=float32)


**Generating queries, keys, and values for multiple heads.**

> Now that we have a way to calculate self-attention, let's actually generate the input queries, keys, and values for multiple heads.

>  each attention head had its own separate set of query, key, and value weights. Each weight matrix was of dimension  d x d/h  where h was the number of heads.




In [42]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.d_model = d_model
    self.num_heads = num_heads

    self.d_head = self.d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(self.d_model, use_bias=False)
    self.wk = tf.keras.layers.Dense(self.d_model, use_bias=False)
    self.wv = tf.keras.layers.Dense(self.d_model, use_bias=False)

    # Linear layer to generate the final output.
    self.dense = tf.keras.layers.Dense(self.d_model)

  def split_heads(self, x):
    batch_size = x.shape[0]

    split_inputs = tf.reshape(x, (batch_size, -1, self.num_heads, self.d_head))
    return tf.transpose(split_inputs, perm=[0, 2, 1, 3])

  def merge_heads(self, x):
    batch_size = x.shape[0]

    merged_inputs = tf.transpose(x, perm=[0, 2, 1, 3])
    return tf.reshape(merged_inputs, (batch_size, -1, self.d_model))

  def call(self, q, k, v, mask):
    qs = self.wq(q)
    ks = self.wk(k)
    vs = self.wv(v)

    qs = self.split_heads(qs)
    ks = self.split_heads(ks)
    vs = self.split_heads(vs)

    output, attn_weights = scaled_dot_product_attention(qs, ks, vs, mask)
    output = self.merge_heads(output)

    return self.dense(output), attn_weights


Feed Forward Neural Network

In [43]:
def feed_forward_network(d_model, hidden_dim):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(hidden_dim, activation='relu'),
      tf.keras.layers.Dense(d_model)
  ])

Decode Block

In [50]:
class DecoderBlock(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, hidden_dim, dropout_rate=0.1):
    super(DecoderBlock, self).__init__()

    self.mhsa1 = MultiHeadAttention(d_model, num_heads)

    self.ffn = feed_forward_network(d_model, hidden_dim)

    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    self.layernorm1 = tf.keras.layers.LayerNormalization()
    self.layernorm2 = tf.keras.layers.LayerNormalization()

  # Note the decoder block takes two masks. One for the first MHSA, another
  # for the second MHSA.
  def call(self, target, training, decoder_mask):
    mhsa_output1, attn_weights = self.mhsa1(target, target, target, decoder_mask)
    mhsa_output1 = self.dropout1(mhsa_output1, training=training)
    mhsa_output1 = self.layernorm1(mhsa_output1 + target)

    ffn_output = self.ffn(mhsa_output1)
    ffn_output = self.dropout2(ffn_output, training=training)
    output = self.layernorm2(ffn_output + mhsa_output1)

    return output, attn_weights


Decoder with Mulitple Layers

In [51]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
               max_seq_len, dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.max_seq_len = max_seq_len

    self.token_embed = tf.keras.layers.Embedding(target_vocab_size, self.d_model)
    self.pos_embed = tf.keras.layers.Embedding(max_seq_len, self.d_model)

    self.dropout = tf.keras.layers.Dropout(dropout_rate)

    self.blocks = [DecoderBlock(self.d_model, num_heads, hidden_dim, dropout_rate) for _ in range(num_blocks)]

  def call(self, target, training, decoder_mask):
    token_embeds = self.token_embed(target)

    # Generate position indices.
    num_pos = target.shape[0] * self.max_seq_len
    pos_idx = np.resize(np.arange(self.max_seq_len), num_pos)
    pos_idx = np.reshape(pos_idx, target.shape)

    pos_embeds = self.pos_embed(pos_idx)

    x = self.dropout(token_embeds + pos_embeds, training=training)

    for block in self.blocks:
      x, weights = block(x, training, decoder_mask)

    return x, weights

Custom loss function to remove effect of padding

In [52]:
def loss_func(targets, logits):
  ce_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

  mask = tf.cast(tf.math.not_equal(targets, 0), tf.float32)

  return ce_loss(targets, logits, sample_weight=mask)

In [59]:
class GPT2Model(tf.keras.Model):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
               max_input_len, dropout_rate=0.1):
    super(GPT2Model, self).__init__()

    self.decoder = Decoder(num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
                           max_input_len, dropout_rate)
    # The final dense layer to generate logits from the decoder output.
    self.output_layer = tf.keras.layers.Dense(target_vocab_size)

  @tf.function
  def train_step(self, inputs):
      loss = 0.

      decoder_input_seq, targets = inputs

      with tf.GradientTape() as tape:

        dec_padding_mask = tf.cast(tf.math.not_equal(decoder_input_seq, 0), tf.float32)
        dec_padding_mask = dec_padding_mask[:, tf.newaxis, tf.newaxis, :]
        target_input_seq_len = len(decoder_input_seq[0])
        look_ahead_mask = tf.linalg.band_part(tf.ones((target_input_seq_len,
                                               target_input_seq_len)), -1, 0)
        dec_mask = tf.minimum(dec_padding_mask, look_ahead_mask)

        logits, _ = self.decoder(decoder_input_seq, True, dec_mask)
        logits =   self.output_layer(logits)
        # The loss is now accumulated through the whole batch
        loss += self.loss(targets, logits)

      # Update the parameters and the optimizer
      variables = self.decoder.trainable_variables
      gradients = tape.gradient(loss, variables)
      self.optimizer.apply_gradients(zip(gradients, variables))

      return {'loss': loss / targets.shape[1]}

  def call(self, target, training):
    logits, _ = self.decoder(decoder_input_seq, True, None)
    logits =   self.output_layer(logits)
    return logits



In [60]:
model = GPT2Model(
    num_blocks = 6,
    d_model = 512,
    num_heads = 4,
    hidden_dim = 1024,
    target_vocab_size = tokenizer.get_vocab_size(),
    max_input_len = 256)

optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_func)

In [61]:
epochs = 12
model.fit(tf_dataset, epochs=epochs)

Epoch 1/12
100/100 [==============================] - 954s 9s/step - loss: 0.0261
Epoch 2/12
100/100 [==============================] - 906s 9s/step - loss: 0.0259
Epoch 3/12
 34/100 [=========>....................] - ETA: 9:59 - loss: 0.0262 

KeyboardInterrupt: ignored